In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import psycopg2
import datetime
import re
import locale
import sys
from itertools import product
import time
import ipycytoscape
from ipycytoscape import CytoscapeWidget, Node, Edge

In [2]:
conn = psycopg2.connect(
    host="localhost",
    database="MF",
    user="dzukas",
    password="dzukas6",
    port=5432
) 

cur = conn.cursor()

In [ ]:
pattern = "ik.*ak.*au$"

cur.execute(f"""
            SELECT word FROM dict
             WHERE word ~ '{pattern}' AND trash is false
""")
nodes = cur.fetchall()
nodes = tuple(node[0] for node in nodes)

cyto = CytoscapeWidget()
# cyto.layout.width  = '1000px'
cyto.layout.height = '5000px'

cyto.set_style([
    {
        'selector': 'node',
        'style': {
            'content': 'data(label)',
            'text-valign': 'center',
            'background-color': '#61bffc'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': '#a3c4dc',
            'width': 2
        }
    }
])

for u in nodes:
    # on peut créer chaque noeud une seule fois, mais ça ne dupliquera pas  
    cyto.graph.add_node(Node(data={"id": str(u), "label": str(u)}))
    # cyto.graph.add_node(Node(data={"id": str(v), "label": str(v)}))
    # cyto.graph.add_edge(Edge(data={"source": str(u), "target": str(v)}))

# cyto.set_layout(name='grid')      # grille
# cyto.set_layout(name='circle')  # cercle
layout_opts = {
    'name': 'grid',                 # grille
    'fit': False,                    # ajuste le zoom pour couvrir tout le widget
    'padding': 50,                  # marge autour du layout
    'rows': 1,                   # nb de lignes
    # 'cols': 5,                   # nb de colonnes
    'avoidOverlap': True,           # empêche le chevauchement
    'avoidOverlapPadding': 10,      # espace entre nœuds
    'spacingFactor': 1.2,           # agrandit/comprime la grille
    'condense': False               # répartit sur tout l’espace disponible
}  # hiérarchique, etc.

cyto

In [9]:
conn.rollback()

In [8]:

cur.execute("""
            SELECT m1.morph, m2.morph
            FROM relations AS mr
            LEFT JOIN (
            SELECT morph, id
            FROM morphs
            ) as m1
            ON mr.m1_id = m1.id
            LEFT JOIN (
            SELECT morph, id
            FROM morphs
            ) as m2
            ON mr.m2_id = m2.id
            WHERE m1.morph = 'rožė' or m2.morph = 'rožė'
            --or m1.morph ~ 'ož' or m2.morph = 'ož' 
""")
edges = cur.fetchall()

cyto = CytoscapeWidget()
# cyto.layout.width  = '1000px'
cyto.layout.height = '1000px'
cyto.set_style([
    {
        'selector': 'node',
        'style': {
            'content': 'data(label)',
            'text-valign': 'center',
            'background-color': '#61bffc'
        }
    },
    {
        'selector': 'edge',
        'style': {
            'line-color': '#a3c4dc',
            'width': 2
        }
    }
])

for u, v in edges:
    # on peut créer chaque noeud une seule fois, mais ça ne dupliquera pas  
    cyto.graph.add_node(Node(data={"id": str(u), "label": str(u)}))
    cyto.graph.add_node(Node(data={"id": str(v), "label": str(v)}))
    cyto.graph.add_edge(Edge(data={"source": str(u), "target": str(v)}))

cyto

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …

In [6]:
import networkx as nx
from pyvis.network import Network

cur.execute("""
            SELECT m1.morph, m2.morph
            FROM relations AS r
            LEFT JOIN (
            SELECT morph, id
            FROM morphs
            ) as m1
            ON r.m1_id = m1.id
            LEFT JOIN (
            SELECT morph, id
            FROM morphs
            ) as m2
            ON r.m2_id = m2.id
            --WHERE m1.morph = 'tiktaktau' or m2.morph = 'tiktaktau' or m1.morph = 'ik.*ak.*au$' or m2.morph = 'ik.*ak.*au$'
""")
edges = cur.fetchall()

G = nx.Graph()
for edge in edges:
    G.add_edge(edge[0], edge[1])

net = Network(notebook=True, cdn_resources='remote', height="100vh", width = "100%")
net.from_nx(G)
net.show("graph.html")

graph.html
